In [ ]:
#@title Setup

!git clone https://github.com/EleutherAI/pushshift_dump_processing
%cd pushshift_dump_processing
%pip install -r requirements.txt

In [ ]:
#@title Mount Your Google Drive - You will need lots of space!

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Verify Google Drive Mount

import os

gd_basepath = "/content/drive/My Drive"
test_file_path = os.path.join(gd_basepath, "foo.txt")
with open(test_file_path, 'w') as f:
  f.write('Hello Google Drive!')
!cat '{test_file_path}'

In [ ]:
#@title Downloads all pushshift dumps from start date - end date
import datetime
from download_pushshift_dumps import main

start_year = 2019 #@param {type:"integer"}
start_month =  1#@param {type:"integer"}

end_year = 2019 #@param {type:"integer"}
end_month =  1#@param {type:"integer"}

start_date = datetime.datetime(start_year, start_month, 1)    
end_date = datetime.datetime(end_year, end_month, 1)

logfile_path = "download_pushshift_dumps.log"
download_directory = "dumps"

main(logfile_path, start_date, end_date, download_directory)

In [ ]:
#@title Parse urls from all downloaded pushshift files

from process_dump_files import main

dumps_dir = "dumps"
urls_output_dir = "urls"

main(dumps_dir, urls_output_dir)

In [ ]:
#@title Copy URL files to Google Drive
colab_instance_id = 5 #@param {type:"integer"}

urls_sub = f"urls{colab_instance_id}"
gd_urls_directory = os.path.join(gd_basepath, urls_sub)
!mkdir '{gd_urls_directory}'
!cp {urls_output_dir}/* '{gd_urls_directory}'
!ls '{gd_urls_directory}'

In [ ]:
#@title Scrapes text from all parsed urls

from scrape_urls import main
import glob

logfile_name = "scrape_urls.log"
process_count = 60
chunk_size = 10000
output_directory = os.path.join(gd_basepath, "scrapes")

all_processed_dumps = glob.glob(f"{gd_urls_directory}/RS_*.txt")
for dump in all_processed_dumps:
  print(dump)
  main(logfile_name, dump, output_directory, chunk_size=chunk_size, 
       process_count=process_count)